# Projeto final módulo 2 - parte bônus

Para o projeto final do módulo 2 (parte bônus), deverá ser feito um sistema de recomendação de livros, utilizando filtragem colaborativa baseada em usuário.

## Os dados

Os dados em anexo na atividade são os seguintes:

* ratings.csv
    * Este conjunto de dados contém informações de avaliações dos usuários para com os livros, onde cada observação é uma avaliação de um usuário para com um livro.
* books.csv
    * Este conjunto de dados contém informações pertinentes ao livro, como o nome do livro, autores, etc.
* to_read.csv
    * Este conjunto de dados contém livros que os usuários salvaram para ler depois.

Para realizar o exercício só é necessário utilizar o arquivo `ratings.csv`, porém o arquivo `books.csv` é necessário para identificar qual o nome do livro que o usuário avaliou. O arquivo `to_read.csv` é opcional para avaliar se o usuário teve interesse em determinado livro.

## Sistemas de recomendação

Separe o conjunto de dados em 2 partes, sendo uma com 500 usuários (sorteados aleatoriamente) que tenham pelo menos 30 avaliações (conjunto de teste) e a outra com os demais (conjunto de treino). A interseção entre os dois conjuntos deve ser vazia.

### Sistema de recomendação baseado em filtragem colaborativa

Crie um sistema de recomendação utilizando filtragem colaborativa baseado em usuário, utilizando apenas o conjunto de treino para modelá-lo. Este sistema de recomendação deve utilizar similaridade de usuários baseado na correlação de Pearson. A fórmula da recomendação é descrita no slide 27 da aula de introdução a sistemas de recomendação, enquanto a similaridade é descrita no slide 24. 

Este sistema vai predizer uma nota para cada par de livro e usuário. Com isso, você deve recomendar para cada usuário os 25 livros com maiores notas indicadas pelo sistema, para cada usuário do conjunto de teste. Em seguida, avalie o desempenho das recomendações respondendo as seguintes perguntas: 

* Quantos usuários se interessaram por pelo menos um livro dos recomendados? 
    * Sugestão: utilize contagem da interseção entre o conjunto de livros recomendados e o conjunto de livros avaliados pelos usuários. Outra possibilidade é utilizar os livros que os usuários salvaram para ler, utilizando então a interseção entre o conjunto dos livros recomendados e o conjunto dos livros salvos para ler.
* Veja a distribuição de quantos livros os usuários se interessaram dos que foram recomendados.
    * Note que essa distribuição deve ir de 0 - 25, tendo em vista que só foram recomendados 25 livros, logo a interseção entre os livros que os usuários tiveram interesse e os recomendados não podem superar a contagem de 25.


In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
#from spicy import stats

df_books = pd.read_csv('https://orionwinter.github.io/datasets/books.csv')
df_ratings = pd.read_csv('https://orionwinter.github.io/datasets/ratings.csv')
df_to_read = pd.read_csv('https://orionwinter.github.io/datasets/to_read.csv')

In [0]:
df_books_ratings = df_ratings.set_index('book_id').join(df_books.set_index('book_id'),'book_id', 'left', lsuffix='_caller', rsuffix='_other')
df_books_ratings = df_books_ratings.reset_index()

In [0]:
df_user_votes_count = df_books_ratings.groupby('user_id', as_index=0).agg({'rating' : 'count'})

df_user_votes_count.columns = df_user_votes_count.columns.str.replace('rating','rating_count')

df_user_votes_count = df_user_votes_count.query('rating_count > 3')

In [0]:
df_books_ratings_without_not_active_users = df_books_ratings.set_index('user_id').join(df_user_votes_count.set_index('user_id'),'user_id','inner')
df_books_ratings_without_not_active_users = df_books_ratings_without_not_active_users.reset_index()

In [16]:
df_books_ratings_summarized = df_books_ratings_without_not_active_users[['user_id','book_id','rating','title']]

df_books_ratings_summarized = df_books_ratings_summarized.groupby(['user_id','book_id','title']).agg({'rating' : 'min'})

df_books_ratings_summarized.sort_values('user_id',ascending=False).head(1)

,,,rating
user_id,book_id,title,
53424,4214,Life of Pi,5


In [0]:
df_books_ratings_summarized = df_books_ratings_summarized.reset_index()

In [0]:
df_users_rating_more_than_30 = df_user_votes_count.query('rating_count > 30')

df_users_rating_more_than_30.sort_values('rating_count',ascending=True)

df_unique_users_rating_more_than_30_votes = df_users_rating_more_than_30['user_id'].drop_duplicates().tolist()

random_500_users_from_the_more_than_30_votes = random.sample(df_unique_users_rating_more_than_30_votes, k=500)

other_users = df_user_votes_count['user_id'].drop_duplicates().tolist()

other_users = np.setdiff1d(other_users,random_500_users_from_the_more_than_30_votes).tolist() 

In [0]:
random_500_users_from_the_more_than_30_votes

In [0]:
filter1 = df_ratings['user_id'].isin(random_500_users_from_the_more_than_30_votes) 

df_ratings_from_the_most_active_users = df_ratings[filter1]

In [0]:
df_ratings_from_the_most_active_users

In [23]:
df_ratings_from_the_most_active_users.sort_values(['user_id','rating'],ascending=False).head()

,book_id,user_id,rating
6798,68,52740,5
17496,175,52740,5
21699,217,52740,5
22599,226,52740,5
25299,253,52740,5


In [0]:
other_users

In [0]:
df_target_user = df_books_ratings_summarized[df_books_ratings_summarized['user_id'] == 29703]
df_target_user = df_target_user[['book_id','rating']]
correlatino_matrix = []

for i in random_500_users_from_the_more_than_30_votes:
  #for j in df_target_user['book_id']:
  df_result = df_ratings_from_the_most_active_users[(df_ratings_from_the_most_active_users['user_id'] == i)]
  df_result = df_result[['book_id','rating']]
  #df_result = df_ratings_from_the_most_active_users[(df_ratings_from_the_most_active_users['user_id'] == i) and (df_ratings_from_the_most_active_users['book_id'] == j)]
  correlation_matrix.append({'user_id': i,'correlation' :df_result.corrwith(df_target_user)}) 
  
df_correlation = pd.DataFrame(correlation_matrix)
df_correlation

In [94]:
df_target_user = df_books_ratings_summarized[df_books_ratings_summarized['user_id'] == 29703]
df_target_user = df_target_user[['book_id','rating']]
df_result = df_ratings_from_the_most_active_users[(df_ratings_from_the_most_active_users['user_id'] == 29703)]
df_result = df_result[['book_id','rating']]
df_result.corrwith(df_target_user)

book_id   NaN
rating    NaN
dtype: float64

In [92]:
df_target_user = df_books_ratings_summarized[df_books_ratings_summarized['user_id'] == 29703]
df_target_user = df_target_user[['book_id','rating']]
df_target_user.sort_values('book_id').head()

,book_id,rating
45649,1,5
45650,3,1
45651,5,5
45652,10,4
45653,11,5


In [93]:
df_result = df_ratings_from_the_most_active_users[(df_ratings_from_the_most_active_users['user_id'] == 29703)]
df_result = df_result[['book_id','rating']]
df_result.sort_values('book_id').head()

,book_id,rating
59,1,5
263,3,1
360,4,5
464,5,5
968,10,4


In [78]:
df_target_user = df_books_ratings_summarized[df_books_ratings_summarized['user_id'] == 7]
df_target_user = df_target_user[['book_id','rating']]
df_target_user

,book_id,rating
0,1519,5
1,3711,5
2,4138,3
3,4588,3


In [67]:
df_books_ratings_summarized

,user_id,book_id,title,rating
0,7,1519,"The Oresteia (Ορέστεια, #1-3)",5
1,7,3711,White Teeth,5
2,7,4138,Naked,3
3,7,4588,Extremely Loud and Incredibly Close,3
4,9,8676,Unlimited Power : The New Science Of Personal ...,4
5,10,5084,My Life in France,2
6,10,5907,The Hobbit,4
7,10,9854,The End of Poverty,4
8,11,9717,The Unbearable Lightness of Being,4
9,14,7604,Lolita,3


In [50]:
i

29703

In [51]:
j

NameError: ignored